If you are running this notebook on Binder, you may need to login to Coiled first.
Run the following line in a cell:
```
!coiled login --workspace class-2025
```

In [1]:
import coiled
from dask.distributed import Client

In [2]:
# Connect to cluster: 
# cluster = coiled.Cluster(name="ds2-highcpu", workspace="class-2025")
cluster = coiled.Cluster(name="ds2-highmem", workspace="class-2025")
client = cluster.get_client()

Output()

Could not get token from client GCP session. This is not a concern unless you're planning to use forwarded GCP credentials on your cluster. The error was: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})


In [3]:
%%time
import dask.array as da
x = da.random.normal(10, 0.1, size=(200000,20000), chunks= (1000,1000))
y = x.mean(axis=0)[::100]
y.compute();
x.nbytes/1e9, y.nbytes/1e9

# Wall times:
# - ds2-highcpu: 10.7 s, 7.38 s, 7.53 s, 7.37 s
# - ds2-highmem: 14.7 s, 10 s, 12 s, 15.8 s, 12.7 s

/Users/gmaze/miniconda3/envs/ds2-coiled-2025-binder/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 10.18 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 1.15 s, sys: 449 ms, total: 1.6 s
Wall time: 38.9 s


(32.0, 1.6e-06)

In [4]:
from intake import open_catalog
catalog_url = 'https://raw.githubusercontent.com/obidam/ds2-2025/main/ds2_data_catalog.yml'
cat = open_catalog(catalog_url)
cat

ds2_data_catalog:
  args:
    path: https://raw.githubusercontent.com/obidam/ds2-2025/main/ds2_data_catalog.yml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


In [5]:
ds = cat["en4"].to_dask()
print("Size of the dataset:", ds.nbytes/1e9,"Gb")
ds

/Users/gmaze/miniconda3/envs/ds2-coiled-2025-binder/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


Size of the dataset: 52.2317975 Gb


/Users/gmaze/miniconda3/envs/ds2-coiled-2025-binder/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


<xarray.Dataset>
Dimensions:                          (depth: 42, time: 832, bnds: 2, lat: 173,
                                      lon: 360)
Coordinates:
  * depth                            (depth) float32 5.022 15.08 ... 5.35e+03
  * lat                              (lat) float32 -83.0 -82.0 ... 88.0 89.0
  * lon                              (lon) float32 1.0 2.0 3.0 ... 359.0 360.0
  * time                             (time) datetime64[ns] 1950-01-16T12:00:0...
Dimensions without coordinates: bnds
Data variables:
    depth_bnds                       (time, depth, bnds) float32 dask.array<chunksize=(1, 42, 2), meta=np.ndarray>
    salinity                         (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    salinity_observation_weights     (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    salinity_uncertainty             (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    temperature                      (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    temperature_observation_weights  (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    temperature_uncertainty          (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    time_bnds                        (time, bnds) datetime64[ns] dask.array<chunksize=(832, 2), meta=np.ndarray>
Attributes: (12/21)
    Conventions:            CF-1.0
    DSD_entry_id:           UKMO-L4UHFnd-GLOB-v01
    GDS_version_id:         v1.7
    contact:                Simon Good - simon.good@metoffice.gov.uk
    creation_date:          2017-04-21 21:12:08.123 -00:00
    easternmost_longitude:  362.5
    ...                     ...
    start_date:             2001-01-01 UTC
    start_time:             00:00:00 UTC
    stop_date:              2001-01-01 UTC
    stop_time:              00:00:00 UTC
    title:                  Temperature and salinity analysis
    westernmost_longitude:  0.5

In [26]:
ds = cat['sea_surface_height'].to_dask()
print("Size of the dataset:", ds.nbytes/1e9,"Gb")
ds

Size of the dataset: 18.1203746 Gb


/Users/gmaze/miniconda3/envs/ds2-coiled-2025/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


<xarray.Dataset> Size: 18GB
Dimensions:    (time: 312, latitude: 720, longitude: 1440, nv: 2)
Coordinates:
  * latitude   (latitude) float32 3kB -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 6kB 0.125 0.375 0.625 ... 359.4 359.6 359.9
  * nv         (nv) int32 8B 0 1
  * time       (time) datetime64[ns] 2kB 1993-01-01 1993-02-01 ... 2018-12-01
Data variables:
    adt        (time, latitude, longitude) float64 3GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    crs        (time) float64 2kB dask.array<chunksize=(1,), meta=np.ndarray>
    err        (time, latitude, longitude) float64 3GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    lat_bnds   (time, latitude, nv) float32 2MB dask.array<chunksize=(1, 720, 2), meta=np.ndarray>
    lon_bnds   (time, longitude, nv) float32 4MB dask.array<chunksize=(1, 1440, 2), meta=np.ndarray>
    sla        (time, latitude, longitude) float64 3GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugos       (time, latitude, longitude) float64 3GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugosa      (time, latitude, longitude) float64 3GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgos       (time, latitude, longitude) float64 3GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgosa      (time, latitude, longitude) float64 3GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/44)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    Note:                            Generated by G. Maze with 31 daily files
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Delayed-Time Level-4 sea su...
    time_coverage_duration:          P1D
    time_coverage_end:               1993-01-31T12:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             1993-01-30T12:00:00Z
    title:                           DT merged all satellites Global Ocean Gr...

In [31]:
%%time
ds['adt'].mean().compute()

TypeError: No numeric data to plot.